## Classify Text into Labels

https://js.langchain.com/docs/tutorials/classification/


Tagging means labeling a document with classes such as:

- sentiment
- language
- style (formal, informal etc.)
- covered topics
- political tendency

![Overview](../../assets/images/classification_workflow.png)


#### Quickstart

* Instantiate the model

In [1]:
// Load environment variables
import * as dotenv from "dotenv";
dotenv.config();
// Check if the OPENAI_API_KEY environment variable is set
if (!process.env.OPENAI_API_KEY) {
  throw new Error("Missing OPENAI_API_KEY environment variable");
}

import { ChatOpenAI } from "@langchain/openai";
const llm = new ChatOpenAI({
  model: "gpt-4o-mini",
  temperature: 0
});

* Let’s specify a Zod schema with a few properties and their expected type in our schema.

In [2]:
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { z } from "zod";

const taggingPrompt = ChatPromptTemplate.fromTemplate(
  `Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
`
);

const classificationSchema = z.object({
  sentiment: z.string().describe("The sentiment of the text"),
  aggressiveness: z
    .number()
    .int()
    .describe("How aggressive the text is on a scale from 1 to 10"),
  language: z.string().describe("The language the text is written in"),
});

// Name is optional, but gives the models more clues as to what your schema represents
const llmWihStructuredOutput = llm.withStructuredOutput(classificationSchema, {
  name: "extractor",
});

In [3]:
const prompt1 = await taggingPrompt.invoke({
    input:
      "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!",
  });
  await llmWihStructuredOutput.invoke(prompt1);

{ sentiment: 'positive', aggressiveness: 1, language: 'Spanish' }


#### Finer control

Specifically, we can define:

- possible values for each property
- description to make sure that the model understands the property
- required properties to be returned

In [4]:
import { z } from "zod";

const classificationSchema2 = z.object({
  sentiment: z
    .enum(["happy", "neutral", "sad"])
    .describe("The sentiment of the text"),
  aggressiveness: z
    .number()
    .int()
    .describe(
      "describes how aggressive the statement is on a scale from 1 to 5. The higher the number the more aggressive"
    ),
  language: z
    .enum(["spanish", "english", "french", "german", "italian"])
    .describe("The language the text is written in"),
});

In [5]:
const taggingPrompt2 = ChatPromptTemplate.fromTemplate(
    `Extract the desired information from the following passage.
  
  Only extract the properties mentioned in the 'Classification' function.
  
  Passage:
  {input}
  `
  );
  
  const llmWithStructuredOutput2 = llm.withStructuredOutput(
    classificationSchema2,
    { name: "extractor" }
  );

In [6]:
const prompt2 = await taggingPrompt2.invoke({
    input:
      "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!",
  });
  await llmWithStructuredOutput2.invoke(prompt2);

{ sentiment: 'happy', aggressiveness: 1, language: 'spanish' }


In [7]:
const prompt3 = await taggingPrompt2.invoke({
  input: "Estoy muy enojado con vos! Te voy a dar tu merecido!",
});
await llmWithStructuredOutput2.invoke(prompt3);

{ sentiment: 'sad', aggressiveness: 5, language: 'spanish' }


In [8]:
const prompt4 = await taggingPrompt2.invoke({
    input: "Weather is ok here, I can go outside without much more than a coat",
  });
  await llmWithStructuredOutput2.invoke(prompt4);

{ sentiment: 'happy', aggressiveness: 1, language: 'english' }
